# **<font color='crimson'>Queries to Related Tables</font>**

---

# **2.1 Relationships Between Tables**

---

In [ ]:
###

In [ ]:
CREATE TABLE author(
    author_id INT PRIMARY KEY AUTO_INCREMENT,
    name_author VARCHAR(50))
;

In [ ]:
###

In [ ]:
INSERT INTO author (name_author)
VALUES
    ('Булгаков М.А.'),
    ('Достоевский Ф.М.'),
    ('Есенин С.А.'),
    ('Пастернак Б.Л.');

In [ ]:
###

In [ ]:
CREATE TABLE book (
    book_id INT PRIMARY KEY AUTO_INCREMENT,
    title VARCHAR(50),
    author_id INT NOT NULL,
    genre_id INT,
    price DECIMAL(8, 2),
    amount INT,
    FOREIGN KEY (author_id) REFERENCES author (author_id),
    FOREIGN KEY (genre_id) REFERENCES genre (genre_id)
);

In [ ]:
###

In [ ]:
CREATE TABLE book (
    book_id INT PRIMARY KEY AUTO_INCREMENT,
    title VARCHAR(50),
    author_id INT,
    genre_id INT,
    price DECIMAL(8, 2),
    amount INT,
    FOREIGN KEY (author_id) REFERENCES author (author_id) ON DELETE CASCADE,
    FOREIGN KEY (genre_id) REFERENCES genre (genre_id) ON DELETE SET NULL
)

In [ ]:
###

In [ ]:
INSERT INTO book (
    title, author_id, genre_id, price, amount)
VALUES
    ('Стихотворения и поэмы', 3, 2, 650.00, 15),
    ('Черный человек', 3, 2, 570.20, 6),
    ('Лирика', 4, 2, 518.99, 2);
SELECT * FROM book;

# **2.2 Selection Queries, Joining Tables**

---

In [ ]:
###

In [ ]:
SELECT
    book.title,
    genre.name_genre,
    book.price
FROM
    book INNER JOIN genre
    ON book.genre_id = genre.genre_id
WHERE book.amount > 8
ORDER BY book.price DESC;

In [ ]:
###

In [ ]:
SELECT genre.name_genre
FROM
    genre LEFT JOIN book
    ON genre.genre_id = book.genre_id
WHERE book.title IS NULL;

In [ ]:
###

In [ ]:
SELECT
    city.name_city,
    author.name_author,
    DATE_ADD('2020-01-01', INTERVAL FLOOR(RAND() * 365) DAY) AS Дата
FROM
    city CROSS JOIN author
ORDER BY name_city, Дата DESC;

In [ ]:
###

In [ ]:
SELECT
    genre.name_genre,
    book.title,
    author.name_author
FROM
    genre INNER JOIN book
    ON genre.genre_id = book.genre_id
    INNER JOIN author
    ON book.author_id= author.author_id
WHERE genre.name_genre  LIKE '%роман%'
ORDER BY title ASC;

In [ ]:
###

In [ ]:
SELECT
    author.name_author,
    SUM(book.amount) AS Количество
FROM
    author LEFT JOIN book
    ON author.author_id = book.author_id
GROUP BY author.name_author
HAVING SUM(book.amount) < 10 OR SUM(book.amount) IS NULL
ORDER BY SUM(book.amount) ASC;

In [ ]:
###

In [ ]:
SELECT author.name_author
FROM
    book INNER JOIN author
    ON author.author_id = book.author_id
    INNER JOIN genre
    ON book.genre_id = genre.genre_id
GROUP BY author.name_author
HAVING COUNT(DISTINCT genre.name_genre) = 1;

In [ ]:
###

In [ ]:
SELECT
    book.title,
    author.name_author,
    genre.name_genre,
    book.price,
    book.amount
FROM book
    INNER JOIN author ON book.author_id = author.author_id
    INNER JOIN genre ON genre.genre_id = book.genre_id
WHERE book.genre_id IN (
    SELECT query_1.genre_id
    FROM
        (SELECT
            book.genre_id,
            SUM(book.amount) AS sum_amount
        FROM book
        GROUP BY book.genre_id) AS query_1
    INNER JOIN
        (SELECT
            book.genre_id,
            SUM(book.amount) AS sum_amount
        FROM book
        GROUP BY book.genre_id
        ORDER BY SUM(book.amount) DESC
        LIMIT 1) AS query_2
    ON query_1.sum_amount  = query_2.sum_amount)
ORDER BY title ASC;

In [ ]:
###

In [ ]:
SELECT
    book.title AS Название,
    supply.author AS Автор,
    book.amount + supply.amount AS Количество
FROM book
    INNER JOIN supply USING (title)
WHERE book.price = supply.price;

In [ ]:
###

The cheapest books of each genre.

In [ ]:
SELECT
    author.name_author,
    book.title,
    book.price,
    genre.name_genre
FROM
    book
    INNER JOIN genre
    ON book.genre_id = genre.genre_id
    INNER JOIN author
    ON book.author_id = author.author_id
WHERE price IN (
    SELECT
        MIN(book.price) AS price
    FROM
        book
        INNER JOIN genre
        ON book.genre_id = genre.genre_id
        INNER JOIN author
        ON book.author_id = author.author_id
    GROUP BY name_genre)
ORDER BY name_author;

# **2.3 Selection Requests, Joining Tables**

---

In [ ]:
###

In [ ]:
UPDATE
    book INNER JOIN author
    ON book.author_id = author.author_id
    INNER JOIN supply
    ON book.title = supply.title
        AND supply.author = author.name_author
SET
    book.price = (
        book.price * book.amount + supply.price * supply.amount) / (
            book.amount + supply.amount),
    book.amount = book.amount + supply.amount,
    supply.amount = 0
WHERE
    book.price <> supply.price;

SELECT * FROM book;
SELECT * FROM supply;

In [ ]:
###

In [ ]:
INSERT INTO author (name_author)
SELECT
    supply.author
FROM author
    RIGHT JOIN supply
    ON author.name_author = supply.author
WHERE author.name_author IS NULL;

In [ ]:
###

In [ ]:
INSERT INTO book (
    title, author_id, price, amount)
SELECT
    supply.title,
    author.author_id,
    supply.price,
    supply.amount
FROM supply
    INNER JOIN author
    ON supply.author = author.name_author
WHERE amount <> 0;

SELECT * FROM book;

In [ ]:
###

In [ ]:
UPDATE book
SET
    book.genre_id = 2
WHERE book.book_id = 10;

UPDATE book
SET
    book.genre_id = (
    SELECT genre.genre_id
    FROM genre
    WHERE genre.name_genre = 'Приключения')
WHERE book.book_id = 11;

SELECT * FROM book;

In [ ]:
###

In [ ]:
DELETE FROM book
WHERE author_id IN (
    SELECT book.author_id
    FROM (SELECT * FROM book) AS book_copy
    GROUP BY book.author_id
    HAVING SUM(book.amount) < 20);

In [ ]:
###

In [ ]:
DELETE FROM genre
WHERE genre.genre_id IN (
    SELECT
        book.genre_id
    FROM book
    GROUP BY book.genre_id
    HAVING COUNT(book.title) < 4);

In [ ]:
###

In [ ]:
DELETE FROM author
USING
    author
    INNER JOIN book
    ON book.author_id = author.author_id
    INNER JOIN genre
    ON book.genre_id = genre.genre_id
WHERE genre.name_genre = 'Поэзия';

In [ ]:
###

Remove those authors from the author and book tables who have books in the book table whose titles consist of two or more words.

In [ ]:
DELETE FROM author
USING
    author
    INNER JOIN book
    ON author.author_id = book.author_id
WHERE book.title LIKE '% %';

# **2.4 The Database 'Online Book Store', Selection Requests**

---

In [ ]:
###

In [ ]:
SELECT
    buy.buy_id,
    book.title,
    book.price,
    buy_book.amount
FROM client
    INNER JOIN buy
        ON client.client_id = buy.client_id
    INNER JOIN buy_book
        ON buy.buy_id = buy_book.buy_id
    INNER JOIN book
        ON buy_book.book_id = book.book_id
WHERE client.name_client = 'Баранов Павел'
ORDER BY buy.buy_id ASC, book.title ASC;

In [ ]:
###

In [ ]:
SELECT
    author.name_author,
    book.title,
    COUNT(buy_book.amount) AS Количество
FROM author
    INNER JOIN book
        ON author.author_id = book.author_id
    LEFT JOIN buy_book
        ON book.book_id = buy_book.book_id
GROUP BY author.name_author, book.title
ORDER BY author.name_author ASC, book.title ASC;

In [ ]:
###

In [ ]:
SELECT
    city.name_city,
    COUNT(buy.buy_id) AS Количество
FROM city
    INNER JOIN client
        ON city.city_id = client.city_id
    INNER JOIN buy
        ON client.client_id = buy.client_id
GROUP BY city.name_city
ORDER BY
    SUM(buy.buy_id) DESC,
    city.name_city ASC;

In [ ]:
###

In [ ]:
SELECT
    buy_step.buy_id,
    buy_step.date_step_end
FROM buy_step
    INNER JOIN step
        ON step.step_id = buy_step.step_id
WHERE
    step.name_step = 'Оплата'
    AND buy_step.date_step_end IS NOT NULL;

In [ ]:
###

In [ ]:
SELECT
    buy.buy_id,
    client.name_client,
    SUM(buy_book.amount * book.price) AS Стоимость
FROM client
    INNER JOIN buy
        ON client.client_id = buy.client_id
    INNER JOIN buy_book
        ON buy.buy_id = buy_book.buy_id
    INNER JOIN book
        ON buy_book.book_id = book.book_id
GROUP BY
    buy.buy_id,
    client.name_client
ORDER BY buy.buy_id;

In [ ]:
###

In [ ]:
SELECT
    buy_step.buy_id,
    step.name_step
FROM step
    INNER JOIN buy_step
        ON step.step_id = buy_step.step_id
WHERE buy_step.date_step_beg IS NOT NULL
    AND buy_step.date_step_end IS NULL;

In [ ]:
###

In [ ]:
SELECT
    buy.buy_id,
    DATEDIFF(
        buy_step.date_step_end,
        buy_step.date_step_beg) AS Количество_дней,
    IF(DATEDIFF(
        buy_step.date_step_end,
        buy_step.date_step_beg) > city.days_delivery,
       ABS(DATEDIFF(
        buy_step.date_step_end,
        buy_step.date_step_beg) - city.days_delivery),
       0) AS Опоздание
FROM city
    INNER JOIN client
        ON city.city_id = client.city_id
    INNER JOIN buy
        ON client.client_id = buy.client_id
    INNER JOIN buy_step
        ON buy.buy_id = buy_step.buy_id
    INNER JOIN step
        ON buy_step.step_id = step.step_id
WHERE step.name_step = 'Транспортировка'
    AND (buy_step.date_step_beg IS NOT NULL
         AND buy_step.date_step_end IS NOT NULL)
ORDER BY buy.buy_id;

In [ ]:
###

In [ ]:
SELECT
    DISTINCT client.name_client
FROM client
    INNER JOIN buy
        ON client.client_id = buy.client_id
    INNER JOIN buy_book
        ON buy.buy_id = buy_book.buy_id
    INNER JOIN book
        ON buy_book.buy_id = book.book_id
    INNER JOIN author
        ON book.author_id = author.author_id
WHERE author.name_author = 'Достоевский Ф.М.'
ORDER BY name_client ASC;

In [ ]:
###

In [ ]:
SELECT
    genre.name_genre,
    SUM(buy_book.amount) AS Количество
FROM genre
    INNER JOIN book
        ON genre.genre_id = book.genre_id
    INNER JOIN buy_book
        ON book.book_id = buy_book.book_id
GROUP BY genre.name_genre
HAVING SUM(buy_book.amount) IN (
    SELECT
        MAX(sum_amount)
    FROM (
        SELECT
            SUM(buy_book.amount) AS sum_amount
        FROM genre
            INNER JOIN book
                ON genre.genre_id = book.genre_id
            INNER JOIN buy_book
                ON book.book_id = buy_book.book_id
        GROUP BY genre.name_genre) AS temp_table);

In [ ]:
###

In [ ]:
SELECT
    YEAR(buy_archive.date_payment) AS Год,
    MONTHNAME(buy_archive.date_payment) AS Месяц,
    SUM(buy_archive.price * buy_archive.amount)AS Сумма
FROM
    buy_archive
GROUP BY Год, Месяц
UNION
SELECT
    YEAR(buy_step.date_step_end) AS Год,
    MONTHNAME(buy_step.date_step_end) AS Месяц,
    SUM(buy_book.amount * book.price) AS Сумма
FROM step
    INNER JOIN buy_step USING(step_id)
    INNER JOIN buy USING(buy_id)
    INNER JOIN buy_book USING(buy_id)
    INNER JOIN book USING(book_id)
WHERE
    buy_step.date_step_end IS NOT NULL
    AND step.name_step = 'Оплата'
GROUP BY Год, Месяц
ORDER BY МЕСЯЦ ASC, Год ASC;

In [ ]:
###

In [ ]:
SELECT
    title,
    SUM(Количество) AS Количество,
    SUM(Сумма) AS Сумма
FROM(
    SELECT
        book.title,
        SUM(buy_archive.amount) AS Количество,
        SUM(buy_archive.amount * buy_archive.price) AS Сумма
    FROM buy_archive
        INNER JOIN book USING(book_id)
    GROUP BY book.title
    UNION
    SELECT
        book.title,
        SUM(buy_book.amount) AS Количество,
        SUM(buy_book.amount * book.price) AS Сумма
    FROM book
        INNER JOIN buy_book USING(book_id)
        INNER JOIN buy USING(buy_id)
        INNER JOIN buy_step USING(buy_id)
        INNER JOIN step USING(step_id)
    WHERE
        buy_step.date_step_end IS NOT NULL
        AND step.name_step = 'Оплата'
    GROUP BY book.title) as temp_query
GROUP BY title
ORDER BY Сумма DESC;

In [ ]:
###

Print the titles of books (indicating the authors) that have never been ordered, sorted alphabetically. Specify the number of available copies in the warehouse.

In [ ]:
SELECT
    book.title,
    author.name_author,
    book.amount
FROM author
    INNER JOIN book USING(author_id)
WHERE book.book_id NOT IN (
    SELECT
        book.book_id
    FROM author
        INNER JOIN book USING(author_id)
        INNER JOIN buy_book USING(book_id)
    GROUP BY
        book.book_id,
        book.title,
        author.name_author)
ORDER BY title ASC;

# **2.5 The Database 'Online Book Store', Correction Requests**

---

In [ ]:
###

In [ ]:
INSERT INTO client (
    name_client, city_id, email)
SELECT
    'Попов Илья', city_id, 'popov@test'
FROM city
WHERE name_city LIKE 'Москва';
SELECT * FROM client;

In [ ]:
###

In [ ]:
INSERT INTO buy (
    buy_description, client_id)
SELECT
    'Связаться со мной по вопросу доставки',
    client_id
FROM client
WHERE client.name_client LIKE 'Попов Илья';
SELECT * FROM buy;

In [ ]:
###

In [ ]:
INSERT INTO buy_book (
    buy_id, book_id, amount)
SELECT 5, book.book_id, 2
FROM book
    INNER JOIN author USING(author_id)
WHERE
    author.name_author = 'Пастернак Б.Л.'
    AND book.title = 'Лирика';

INSERT INTO buy_book (
    buy_id, book_id, amount)
SELECT 5, book.book_id, 1
FROM book
    INNER JOIN author USING(author_id)
WHERE
    author.name_author = 'Булгаков М.А.'
    AND book.title = 'Белая гвардия';

In [ ]:
###

In [ ]:
UPDATE
    book INNER JOIN buy_book USING(book_id)
SET
    book.amount = book.amount - buy_book.amount
WHERE
    buy_book.buy_id = 5;

In [ ]:
###

In [ ]:
CREATE TABLE buy_pay AS
SELECT
    book.title,
    author.name_author,
    book.price,
    buy_book.amount,
    book.price * buy_book.amount AS Стоимость
FROM author
    INNER JOIN book USING(author_id)
    INNER JOIN buy_book USING(book_id)
WHERE buy_book.buy_id = 5
ORDER BY book.title;

SELECT * FROM buy_pay;

In [ ]:
###

In [ ]:
CREATE TABLE buy_pay AS
SELECT
    buy_book.buy_id,
    SUM(buy_book.amount) AS Количество,
    SUM(book.price * buy_book.amount) AS Итого
FROM buy_book
    INNER JOIN book USING(book_id)
WHERE buy_book.buy_id = 5;

In [ ]:
###

In [ ]:
INSERT INTO buy_step (
    buy_id, step_id)
SELECT
    buy.buy_id,
    step.step_id
FROM buy
    CROSS JOIN step
WHERE buy.buy_id = 5;

In [ ]:
###

In [ ]:
UPDATE buy_step
    INNER JOIN step USING(step_id)
SET buy_step.date_step_beg = '2020-04-12'
WHERE
    buy_step.buy_id = 5
    AND step_id IN (
        SELECT step_id
        FROM step
        WHERE name_step = 'Оплата'
);

SELECT * FROM buy_step;

In [ ]:
###

In [ ]:
UPDATE buy_step
    INNER JOIN step USING(step_id)
SET buy_step.date_step_end = '2020-04-13'
WHERE
    buy_step.buy_id = 5
    AND step_id IN (
        SELECT step_id
        FROM step
        WHERE name_step = 'Оплата'
);

UPDATE buy_step
    INNER JOIN step USING(step_id)
SET buy_step.date_step_beg = '2020-04-13'
WHERE
    buy_step.buy_id = 5
    AND step_id IN (
        SELECT step_id
        FROM step
        WHERE name_step = 'Упаковка'
);

In [ ]:
###

It is necessary to find out at what stage all unfinished orders are now.

Create a step_now table with the Name of the client, the title of the book, the date of the current stage and the name of the stage. Sort the dates in ascending order.

In [ ]:
CREATE TABLE step_now AS
SELECT
    client.name_client,
    book.title,
    buy_step.date_step_beg AS date,
    step.name_step
FROM book
    INNER JOIN buy_book USING(book_id)
    INNER JOIN buy USING(buy_id)
    INNER JOIN buy_step USING(buy_id)
    INNER JOIN step USING(step_id)
    INNER JOIN client USING(client_id)
WHERE
    date_step_beg IS NOT NULL
    AND date_step_end IS NULL
ORDER BY date ASC;

SELECT * FROM step_now;